# Data Journalism Lesson 10: Cleaning text

Clean up your data with text cleaning.

In [ ]:
# --- Load Data and Pre-calculate ---
import pandas as pd

data_url = "../_static/plane-crashes-cleaning/crashes-for-cleaning.csv"
crashes = pd.read_csv(data_url)

# Calculate initial counts for glue variables
totalcities_df = crashes.groupby(['City', 'State']).size().reset_index(name='n')
totalcities_count = len(totalcities_df)

crashes_temp = crashes.copy()
crashes_temp['clean_city'] = crashes_temp['City'].str.title()
cleancities_df = crashes_temp.groupby(['clean_city', 'State']).size().reset_index(name='n')
cleancities_count = len(cleancities_df)

titlechange = totalcities_count - cleancities_count

In [ ]:
# --- Glue variables ---
from myst_nb import glue

glue("totalcities_count", totalcities_count, display=False)
glue("cleancities_count", cleancities_count, display=False)
glue("titlechange", titlechange, display=False)

## The Goal

In this lesson, you'll learn about advanced data cleaning techniques using text-cleaning functions and concepts similar to those found in tools like OpenRefine. By the end of this tutorial, you'll understand how to use some text manipulation functions to merge similar text entries. You'll learn to critically evaluate the results. This skill is crucial for data journalists who often work with messy, real-world datasets that require careful standardization before analysis.

## What is Data Journalism?

Gather 'round kids and let me tell you a tale about your author. In college, your author got involved in a project where he mapped crime in the city, looking specifically in the neighborhoods surrounding campus. This was in the mid 1990s. Computers were under powered. Tools were pretty primitive. I was given a database of nearly 50,000 calls for service.

And then I learned that addresses were not stored in a standard way. However the officer wrote it down, that's how it was recorded.

What did that mean?

It meant the Lincoln Police Department came up with dozens of ways to say a single place. And since the mapping software needed the addresses to be in a specific form, I had to fix them. For example, I will go to my grave knowing that Lincoln High School's street address is 2229 J Street. Police officers wrote down LHS, L.H.S., Lincoln HS, Lincoln H.S., LHS (J Street), 2229 J, 2229 J ST, St., Street and on and on and on. That one was relatively easy. A local convenience store chain, with eight locations around the city, was harder. I had to use the patrol district to locate them.

It took me four months to clean up more than 30,000 unique addresses and map them.

I did most of it by hand -- pulling up records in Excel and hand editing them. It was miserable.

What you're going to learn today are things I wish I knew then. Once you know the basics, outside of this tutorial you should look into learning how to use OpenRefine. OpenRefine is a series of tools -- algorithms -- that find small differences in text and helps you fix them quickly. How OpenRefine finds those small differences is through something called clustering. Clustering is where, through algorithms, it finds small meaningless differences between two rows of data -- "Omaha" vs "omaha", for example -- and surmises that they are the same thing.

I tell you this because if I had OpenRefine, it would have taken me a week, not four months. Every time I talk about OpenRefine, I remember this, and I get mad.

## The Basics

Data cleaning is a series of logic puzzles. The logic is very simple: I have a goal. Most often that goal is to have all the like items together. The problem is always that someone did something to prevent that. The logic game you are playing is doing things to your text until all the like things match. No matter what it is you need done to the text, someone, somewhere has screwed that up before and a programmer wrote a function to fix it.

Let's dig into some of them, using a version of the plane crash data we've worked with before. This time it's a special cut of the data -- it's national data, but a limited number of columns. It just makes it easier on everyone to have a smaller dataset.

We're going to start, like every tutorial, with libraries. In this one, we're going to use `pandas` for data manipulation. We'll also explore OpenRefine for cleaning messy text.

For now, we're going to solve some easy problems using tools that work with text.

In [ ]:
import pandas as pd

Now we need data. Because this is national data, there's no state name to fill in. You can just run this code chunk.

In [ ]:
data_url = "../_static/plane-crashes-cleaning/crashes-for-cleaning.csv"
crashes = pd.read_csv(data_url)
crashes.head()

This is the same data from the National Transportation Safety Board of airplane crash investigations since 2019.

Let's take a quick look at it.

In [ ]:
df_head = crashes.____()
print(df_head)

We're specifically going to look at city names. In Data Smells, we learned that we need to look carefully at the data we're going to be relying on. How good of a job has the NTSB done in writing in city names in their database? Quick way? Let's do a simple group by and count the number of cities and states.

In [ ]:
city_state_counts = crashes.groupby([____, ____]).count()
print(city_state_counts)

Doing this, you get {glue:text}`totalcities_count` unique combinations of City and State. Does that mean there's that many different places where plane crashes happened? Without leaving the first page -- the first few results -- should show you the first problem. Notice how the first few rows of cities are all capitalized? Notice one of them is Atlanta, Georgia, home of the busiest airport in the world? What do you want to bet there's a not-all-caps version of Atlanta, Georgia in the data? Don't take that bet. Of course there is.

### Exercise 1: Casing

How do we fix this? Well, you could go through and hand edit the first few cities for every letter of the alphabet, but one thing to keep in mind is that a programmer has already had to deal with this and wrote a function.

Pandas Series have a `.str` accessor that provides many string manipulation functions. Among the more useful are the functions that change text to a specific case. For example, want all uppercase letters? There's `.str.upper()` that does that. And if you guessed there's `.str.lower()` that does the opposite, good on you, mate. You're right.

Neither of those solves our problem. What we need is `.str.title()` which converts things to title case -- aka the first letter of each word is capitalized. Now, is this a perfect solution? Not if you have a city called McDonalds or something like that. MCDONALDS will get turned into Mcdonalds, not McDonalds, using `.str.title()`. But chances are, you have very few of those. And if you have a few, you can fix the vast majority with `.str.title()` and then clean up the few remaining later.

How do we use `.str.title()`? As a rule, we don't want to overwrite original data, so we're going to use `assign` to create a new column and populate it with our new data. So let's make `clean_city` and use `.str.title()` making sure to apply it to the column we're changing to title case. Hint: ATLANTA is our problem. What column did that come from?

In [ ]:
title_cased_counts = (crashes
 .assign(clean_city = crashes[____].str.____())
 .groupby(['clean_city', 'State'])
 .size()
 .reset_index(name='n')
)
print(title_cased_counts)

Doing this, you now get {glue:text}`cleancities_count` unique combinations of City and State. That means we changed the case on cities, then grouped and counted them and {glue:text}`titlechange` cities melded into their properly-cased home.

### Exercise 2: Misspellings, one at a time

If the data gods are smiling upon you, you'll have no spelling errors in your data. It is a good day when they do smile on you, but today is not that day. You can go diving in and find this if you want, or you can trust me that the NTSB misspelled Albuquerque, New Mexico. No shame -- I had to look it up -- but one time in this data, it's Alburquerque, not Albuquerque.

If you only have one of those, it's pretty easy to fix with a search and replace. How do you do that in Python with pandas? With the `.str.replace()` method. It takes a pretty simple and logical input: What pattern are you looking for? That's first. What do you want to replace it with? That's second. You apply this method to the Series (column) you want to modify.

We're going to keep editing `clean_city` until we get what we need. Doing that means we can just chain our new fix onto the previous one. Remember: we're looking for Alburquerque and we want to replace it with Albuquerque. And what column are we going to find that misspelling? It's a bit of a trick question -- remember we're trying to work on the `clean_city` column that is the home of all of our changes from the previous step.

In [ ]:
replaced_counts = (crashes
 .assign(clean_city = crashes['City'].str.title()) # Start with title case
 .assign(clean_city = lambda df: df['clean_city'].str.replace("____", "____", regex=False)) # Apply replacement
 .groupby(['clean_city']) # Grouping just by city now for simplicity
 .size()
 .reset_index(name='n')
)
print(replaced_counts)

And just like that, our count goes down by one because we fixed Alburquerque ... er, Albuquerque.

Note: You can chain `.str.replace()` calls or use more complex logic if you have multiple typos. For example: In Nebraska, there's a city named Fremont that is spelled by the NTSB "Freemont" that you could fix by adding another `.assign(clean_city = lambda df: df['clean_city'].str.replace('Freemont', 'Fremont', regex=False))` line. Rinse and repeat as necessary, if you follow the wisdom of a shampoo bottle.

## OpenRefine

As you've seen, Python and `pandas` are very powerful tools for data journalism. But, there are also times when the easiest and best way to get what we need out of data is to use another tool. This is one of those times. 

OpenRefine is a piece of software that lets you filter and clean messy data. It can do a lot, but it's especially useful for finding misspellings and inconsistencies in text. We're going to use it to clean up the city names in our dataset.

```{warning}
Never trust an algorithm to get it 100 percent right. You are responsible for everything you publish. Always check the output.
```

First, you'll want to [download OpenRefine](https://openrefine.org/download) and install it to your computer. When you launch it, you should see a screen like this:

```{image} ../figures/ch10_i1.png
:alt: instructions
:align: center
```

Now, we need to load our data into the tool. The data we're working with is stored at this URL: [https://the-art-of-data-journalism.github.io/tutorial-data/plane-crashes-cleaning/crashes-for-cleaning.csv](https://the-art-of-data-journalism.github.io/tutorial-data/plane-crashes-cleaning/crashes-for-cleaning.csv). You can either download the file to your computer and load it into OpenRefine, or you can paste the URL directly into the "Web Addresses" option. 

Next, you'll see this screen. Click the "Create project" button.

```{image} ../figures/ch10_i2.png
:alt: instructions
:align: center
```

Now we're ready to clean up our city names. The algorithm we're going to use looks at what you've given it -- the data -- and creates a different version of each thing in your column containing only the most important parts called the key (by lowercasing, removing punctuation/whitespace, sorting words). Then, it merges entries with the same key.

It's my experience that this fingerprinting approach is quite conservative among clustering algorithms. It's right far more than it's wrong -- not even close. Some algorithms you can work with are *really* stretching to match things together. Not so with fingerprinting.

To run it, click the arrow by the "City" column, then "Edit cells", then "Cluster and edit...":

```{image} ../figures/ch10_i4.png
:alt: instructions
:align: center
```

This will open a menu with some options for the algorithm we're going to run. Leave those as-is, and press the "Cluster" button. 

OpenRefine will then look through the city names and find ones that are probably referring to the same place. It gives us the option to review its work and, if it's right, merge inconsistencies. So for example, we can decide to merge these inconsistent spellings and capitalizations of "Fairbanks":

```{image} ../figures/ch10_i5.png
:alt: instructions
:align: center
```

Once you've reviewed OpenRefine's suggestions and are happy with the new values, you can select the "Merge selected & Close" option. This will edit all those inconsistent values, cleaning up your dataset. You can then export the cleaned version for more analysis and cleaning in Python. 

The truth is all text-cleaning jobs are unique. Every dataset has their own problems and you have to solve them. Think of this skill like learning how to play music. I've shown you a few chords. Every text-cleaning job is just playing the chords in a different order, a different combination to solve the problem. And there's a lot more chords out there to play.

How do you fix this city matching problem better? Combining the chords. I'd fix the case first, then maybe standardize abbreviations like "St " (Saint SPACE) to "St. " (Saint PERIOD SPACE) using `.str.replace()`. That gives the fingerprinting algorithm less variation to deal with. Applying pre-processing steps before fingerprinting often yields better results.

## The Recap

Throughout this lesson, you've explored powerful tools for cleaning and standardizing messy text data in Python using pandas. You've learned how to use `.str.title()` for casing, `.str.replace()` for specific substitutions, and how to use OpenRefine for broader text normalization. You've practiced different merging techniques and seen how they can dramatically reduce the number of unique entries in a dataset. However, you've also learned the critical importance of verifying the results of automated cleaning processes. Remember, while these tools can greatly speed up your data cleaning workflow, they should be used judiciously and always with careful human oversight to ensure the accuracy and integrity of your data.

## Terms to Know

- **OpenRefine**: A standalone application for data cleaning that uses clustering algorithms to identify similar text entries.
- **Clustering**: A technique used to group similar data points together based on certain criteria or algorithms.
- **Fingerprinting (Key Collision)**: A text normalization method that extracts key parts of strings (e.g., by lowercasing, removing punctuation, sorting words) to identify and merge similar entries.
- **`.str` accessor**: A pandas feature for Series containing strings, providing access to vectorized string manipulation methods (e.g., `.str.lower()`, `.str.title()`, `.str.replace()`).
- **`.str.title()`**: Capitalizes the first letter of each word in a string Series.
- **`.str.replace()`**: Replaces occurrences of a pattern or literal string with a replacement string in a Series.
- **`.apply()`**: A pandas method to apply a function along an axis of a DataFrame or to a Series.
- **`.assign()`**: A pandas DataFrame method to add new columns or overwrite existing ones, often used with lambda functions for cleaner chaining.
- **Text Normalization**: The process of transforming text into a single, canonical form (e.g., consistent casing, punctuation, spacing).